In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [24]:
from kdg import kdf
from kdg.utils import get_ece
#import openml
import multiprocessing
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import cohen_kappa_score
import os
from kdg.utils import generate_gaussian_parity, pdf, hellinger, gaussian_sparse_parity
from sklearn.utils.validation import check_array, check_is_fitted, check_X_y
import keras
from keras import layers
import tensorflow as tf



In [4]:
n_estimators = 500
sample_size = np.logspace(
        np.log10(10),
        np.log10(5000),
        num=10,
        endpoint=True,
        dtype=int
)

In [6]:
for sample in sample_size:
    print(sample)

10
19
39
79
158
315
629
1256
2506
4999


In [7]:
sample = 10

In [38]:

    network = tf.keras.Sequential()
    network.add(tf.keras.layers.Dense(3, activation='relu', input_shape=(2,)))
    network.add(tf.keras.layers.Dense(3, activation='relu'))
    network.add(tf.keras.layers.Dense(units=2, activation = 'softmax'))

    X, y = generate_gaussian_parity(sample, cluster_std=0.5)
    X_test, y_test = generate_gaussian_parity(1000, cluster_std=0.5)
    p = np.arange(-1,1,step=0.006)
    q = np.arange(-1,1,step=0.006)
    xx, yy = np.meshgrid(p,q)
    grid_samples = np.concatenate(
            (
                xx.reshape(-1,1),
                yy.reshape(-1,1)
            ),
            axis=1
    ) 

In [26]:
X

array([[ 0.35470319, -0.17767971],
       [-0.19926039,  0.56886368],
       [-0.42184798, -0.57556991],
       [-0.57191331, -1.01923075],
       [ 0.56082937, -0.40302382],
       [-0.38298914,  0.49057148],
       [-0.66517986, -0.62874627],
       [-0.29986116,  0.21077361],
       [ 1.30294519, -0.00348044],
       [ 0.57402824, -1.11762278]])

In [27]:
y

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [28]:
cov_type = 'full'

In [29]:
criterion = 'None'

In [39]:
compile_kwargs = {
            "loss": "categorical_crossentropy",
            "optimizer": tf.keras.optimizers.Adam(3e-4)
            }
fit_kwargs = {
            "epochs": 100,
            "batch_size": 32,
            "verbose": False
            }

In [45]:
opt = tf.keras.optimizers.Adam(3e-4)

In [49]:
        X, y = check_X_y(X, y)
        labels = np.unique(y)

        network.compile(loss='categorical_crossentropy', optimizer=opt)

        

In [51]:
        network.fit(X, tf.keras.utils.to_categorical(y), epochs = 100, batch_size = 32)
        feature_dim = X.shape[1]

2021-10-07 12:54:36.359711: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
1/1 [==============================] - 0s 390ms/step - loss: 0.6705
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6703
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6700
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6698
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6696
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6693
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6691
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6688
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6686
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6684
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6681
Epoch 12/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6679
Epoch 13/100
1/1 [==============================] - 0s 2ms/

In [55]:
polytope_means = {}
polytope_cov = {}

In [57]:
        for label in labels:
            polytope_means[label] = []
            polytope_cov[label] = []

            X_ = X[np.where(y==label)[0]]
            

NameError: name '_get_polytopes' is not defined

In [ ]:
polytopes = _get_polytopes(X_)[0]

In [63]:
        polytope_memberships = []
        last_activations = X
        total_layers = len(network.layers)

        for layer_id in range(total_layers):
            print(layer_id)
            weights, bias = network.layers[layer_id].get_weights()
            print("Weights: ", weights, "\n Bias: ", bias)
            preactivation = np.matmul(last_activations, weights) + bias
            print("\n Preactivation: ", preactivation)
            if layer_id == total_layers - 1:
                binary_preactivation = (preactivation > 0.5).astype('int')
            else:
                binary_preactivation = (preactivation > 0).astype('int')
            print("Binary Preactivation: ", binary_preactivation)
            polytope_memberships.append(binary_preactivation)
            last_activations = preactivation * binary_preactivation
            print("Last_Activations: ", last_activations)
        

0
Weights:  [[-0.48429853  0.16967283  0.46321642]
 [ 0.12936315 -0.3152606   0.37111613]] 
 Bias:  [ 0.0263411  -0.02107585  0.02876852]

 Preactivation:  [[ 0.19316466 -0.01507143 -0.24894317]
 [-0.56373615 -0.0155559   0.9620187 ]
 [ 0.69883378 -0.43868858 -0.28079765]
 [ 0.02827443  0.19013682 -0.36151479]
 [-0.57168339  0.64983249 -0.24505764]
 [ 0.02950488 -0.09045003  0.1508864 ]
 [ 0.48168777 -0.34688578 -0.10193336]
 [-0.29066943  0.39471861 -0.22664913]
 [-0.02194037  0.02189867  0.02717678]
 [-0.30197036  0.00314028  0.50925445]]
Binary Preactivation:  [[1 0 0]
 [0 0 1]
 [1 0 0]
 [1 1 0]
 [0 1 0]
 [1 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 1]
 [0 1 1]]
Last_Activations:  [[ 0.19316466 -0.         -0.        ]
 [-0.         -0.          0.9620187 ]
 [ 0.69883378 -0.         -0.        ]
 [ 0.02827443  0.19013682 -0.        ]
 [-0.          0.64983249 -0.        ]
 [ 0.02950488 -0.          0.1508864 ]
 [ 0.48168777 -0.         -0.        ]
 [-0.          0.39471861 -0.        ]
 [-0.   

In [65]:
print(polytope_memberships)

[array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 1, 0],
       [0, 1, 0],
       [1, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 1]]), array([[1, 1, 1],
       [0, 1, 0],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 0]]), array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])]


In [68]:
concatenated = np.concatenate(polytope_memberships, axis = 1)
print("Concatenated: \n", concatenated)

Concatenated: 
 [[1 0 0 1 1 1 0 0]
 [0 0 1 0 1 0 0 0]
 [1 0 0 1 1 1 0 0]
 [1 1 0 0 1 1 0 0]
 [0 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 1 1 1 0 0]
 [0 1 0 0 0 1 0 0]
 [0 1 1 0 1 1 0 0]
 [0 1 1 0 1 0 0 0]]


In [72]:
num_nodes = np.shape(concatenated)[1]
print(np.shape(concatenated))
print(np.shape(concatenated)[1])

(10, 8)
8


In [73]:
2 ** np.arange(0, num_nodes)

array([  1,   2,   4,   8,  16,  32,  64, 128])

In [77]:
np.tensordot(concatenated, 2 ** np.arange(0, num_nodes), axes = 1)

array([57, 20, 57, 51,  2, 29, 57, 34, 54, 22])

In [78]:
[np.tensordot(concatenated, 2 ** np.arange(0, num_nodes), axes = 1)]

[array([57, 20, 57, 51,  2, 29, 57, 34, 54, 22])]

In [76]:
polytope_memberships = [np.tensordot(concatenated, 2 ** np.arange(0, num_nodes), axes = 1)]



In [79]:
polytopes = polytope_memberships[0]#_get_polytopes(X_)[0] #Why is zero needed? Can't we just take it out of array? 

In [61]:
X

array([[-0.40824072, -0.23876052],
       [ 1.41752352,  0.74540011],
       [-1.20848882,  0.67425176],
       [-0.21366528, -0.78495655],
       [ 0.77826123, -1.70924809],
       [ 0.06101886,  0.25289373],
       [-0.77567827,  0.61599326],
       [ 0.35303414, -1.12888883],
       [ 0.073907  , -0.09653763],
       [ 0.76776832,  0.33639883]])

In [62]:
total_layers

3

In [80]:
polytopes

array([57, 20, 57, 51,  2, 29, 57, 34, 54, 22])

In [81]:
polytope = polytopes

In [83]:
                idx = np.where(polytopes==polytope)[0]
                
                if len(idx) == 1:
                    print("Only 1 element") #Is this possible?
                    
                if criterion == None:
                    gm = GaussianMixture(n_components=1, covariance_type=self.covariance_types, reg_covar=1e-4).fit(X_[idx])
                    self.polytope_means[label].append(
                            gm.means_[0]
                    )
                    tmp_cov = gm.covariances_[0]

In [84]:
idx

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
# PENDING: Other covariance + criterion types 